In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sn

In [38]:
# UPLOADING THE MAIN DATASET
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/dataset.csv', low_memory = False)

df['age'] = np.where(((df.age < 18)|(df.age>65)),np.nan,df.age)
df['exp'] = np.where(((df.exp < 0)|(df.exp>47)),np.nan,df.exp)
df = df.dropna(axis = 'rows', subset = ['age','exp'])

df['ln_age'] = np.log(df.age)
df['0'] = 0
df['ln_exp'] = np.log(df.exp)
df['ln_exp'] = df[['ln_exp','0']].max(axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [40]:
# PREPARING BIRTH PLACE REGION DATAFRAME
bp = pd.read_csv('C:/Users/sgulbin/Work/Analysis/Платежеспособность/data_lib/birthplaces_lib.csv')
bp = bp[['PassportBirthPlace','country','region']]
bp = bp.replace('None', np.nan)
bp = bp.dropna(axis='rows')
bp = bp.drop_duplicates(subset = ['PassportBirthPlace'])

# BIRTH PLACES LIB
bp_clsfied = pd.read_csv('C:/Users/sgulbin/Work/Analysis/Платежеспособность/data_lib/birthplaces_classified_lib.csv')

# COUNTRIES
countries = bp_clsfied.drop_duplicates(subset = 'bp_country')
countries = pd.concat([countries.iloc[1:2],countries.iloc[3:]], axis='rows')
countries = countries[['bp_country', 'bp_region_group_detailed']]

# MERGING
bp = pd.merge(bp, bp_clsfied, left_on = 'region', right_on = 'bp_region_group_detailed', how = 'left')
bp = pd.merge(bp, countries, left_on = 'country', right_on = 'bp_country', how = 'left')

bp['bp_region_group_detailed'] = np.where(pd.isnull(bp.bp_region_group_detailed_x) == True,bp.bp_region_group_detailed_y,\
                                            bp.bp_region_group_detailed_x)
bp = bp[['PassportBirthPlace', 'bp_region_group_detailed']]

In [41]:
# MERGING
df = pd.merge(df, bp, left_on = 'birth_place', right_on = 'PassportBirthPlace', how = 'left')

In [42]:
# DEFINING X AND Y
X = df[['sex', 'ln_age', 'ln_exp', 'bp_region_group_detailed', 'kbm_grouped','license_category_grouped']]
X = X.fillna('NaN')

for target in ['accidents_count', 'count_0_20', 'count_20_40', 'count_40_100', 'count_100']:
    
    # CONVERTING % VALUES INTO BOOLEAN
    df['default_user'] = np.where(df[target] > 0,1,0)                  
    y = df['default_user']
        
    # SPLITTING DATASET INTO TRAIN AND TEST SAMPLES
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # ONE-HOT ENCODING VARIABLES
    column_trans = make_column_transformer((StandardScaler(), list(x_train.loc[:, x_train.dtypes == float].columns)),
                                           (OneHotEncoder(), list(x_train.loc[:, x_train.dtypes == object])),\
                                            remainder = 'passthrough')
    # PIPELINE SET UP
    logreg = LogisticRegression(solver = 'lbfgs', class_weight = 'balanced', max_iter = 1000)
    model = make_pipeline(column_trans, logreg)
    _ = model.fit(x_train,y_train)
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    roc_auc_train = roc_auc_score(y_train,y_pred_train)
    roc_auc_test = roc_auc_score(y_test,y_pred_test)
    print(target,';','train:',roc_auc_train,';','test:',roc_auc_test)

accidents_count ; train: 0.6641994089809309 ; test: 0.6585497332080867
count_0_20 ; train: 0.6691406036539272 ; test: 0.6500496559478128
count_20_40 ; train: 0.6949435191023937 ; test: 0.6288542558316682
count_40_100 ; train: 0.7024246845721843 ; test: 0.6451334657756698
count_100 ; train: 0.7285081283156214 ; test: 0.6679064741688121


In [85]:
columns = ['sex', 'age_grouped', 'exp_grouped', 'other_than_B', 'kbm_grouped', 'Region_group', 'accidents_fact']
ds = df[columns]
ds['Region_group'] = ds['Region_group'].fillna(0)
ds['kbm_grouped'] = ds['kbm_grouped'].fillna(0)
ds['age_grouped'] = ds['age_grouped'].replace('50+',50)
ds = ds.dropna(subset=['age_grouped'], how = 'any')
ds['age_grouped'] = ds['age_grouped'].astype(str).astype(int)
ds['exp_grouped'] = ds['exp_grouped'].replace('32+',32)
ds['exp_grouped'] = ds['exp_grouped'].astype(str).astype(int)
oh_sex = pd.get_dummies(ds['sex'])
oh_lcns = pd.get_dummies(ds['other_than_B'])
oh_lcns.columns = ['only_B', 'other_than_B']
oh_rgn = pd.get_dummies(ds['Region_group'])
oh_rgn.columns = oh_rgn.columns = ['no_region']+list(oh_rgn.columns[1:])
oh_kbm = pd.get_dummies(ds['kbm_grouped'])
oh_kbm.columns = oh_kbm.columns = ['no_kbm']+list(oh_kbm.columns[1:])
ds = ds.drop(['sex', 'other_than_B', 'Region_group', 'kbm_grouped'], axis = 1)
ds = ds.join(oh_sex)
ds = ds.join(oh_lcns)
ds = ds.join(oh_rgn)
ds = ds.join(oh_kbm)
y = ds['accidents_fact']
ds = ds.drop(['accidents_fact'], axis = 1)
X = ds
X.info()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753714 entries, 0 to 753774
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   age_grouped        753714 non-null  int32
 1   exp_grouped        753714 non-null  int32
 2   F                  753714 non-null  uint8
 3   M                  753714 non-null  uint8
 4   only_B             753714 non-null  uint8
 5   other_than_B       753714 non-null  uint8
 6   no_region          753714 non-null  uint8
 7   Дальневосточный    753714 non-null  uint8
 8   Приволжский        753714 non-null  uint8
 9   Северо-Западный    753714 non-null  uint8
 10  Северо-Кавказский  753714 non-null  uint8
 11  Сибирский          753714 non-null  uint8
 12  Уральский          753714 non-null  uint8
 13  Центральный        753714 non-null  uint8
 14  Южный              753714 non-null  uint8
 15  no_kbm             753714 non-null  uint8
 16  0.5+               753714 non-null  ui

In [86]:
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
features = [['age_grouped','exp_grouped'],['F','M'],['only_B', 'other_than_B'],\
            ['no_region','Дальневосточный','Приволжский','Северо-Западный','Северо-Кавказский', 'Сибирский', 'Уральский', 'Центральный', 'Южный'],\
            ['0.5+','0.7+','0.8+','0.9+','1','1.4+','2.3+']]
X = X[['age_grouped','exp_grouped','0.5+','0.7+','0.8+','0.9+','1','1.4+','2.3+','F','M']]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
model = XGBRegressor()

model.fit(train_X, train_y)
val_predictions = model.predict(val_X)
val_roc_auc = roc_auc_score(val_y, val_predictions) 
print(val_roc_auc)
# tree.plot_tree(model)
# tree.plot_tree(model,
#                feature_names = X.columns, 
#                filled = True)

0.6763108726910192


In [73]:
# linear regression feature importance
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)
# define the model
# model = DecisionTreeRegressor()
model = XGBRegressor()
# fit the model
model.fit(X, y)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.04762
Feature: 1, Score: 0.06298
Feature: 2, Score: 0.05945
Feature: 3, Score: 0.00000
Feature: 4, Score: 0.06045
Feature: 5, Score: 0.00000
Feature: 6, Score: 0.04415
Feature: 7, Score: 0.04885
Feature: 8, Score: 0.05262
Feature: 9, Score: 0.03585
Feature: 10, Score: 0.04398
Feature: 11, Score: 0.04260
Feature: 12, Score: 0.04066
Feature: 13, Score: 0.03783
Feature: 14, Score: 0.04083
Feature: 15, Score: 0.06572
Feature: 16, Score: 0.03706
Feature: 17, Score: 0.02916
Feature: 18, Score: 0.03053
Feature: 19, Score: 0.06340
Feature: 20, Score: 0.07476
Feature: 21, Score: 0.02725
Feature: 22, Score: 0.05422


In [83]:
features = [['age_grouped','exp_grouped'],['F','M'],['only_B', 'other_than_B'],\
            ['no_region','Дальневосточный','Приволжский','Северо-Западный','Северо-Кавказский', 'Сибирский', 'Уральский', 'Центральный', 'Южный'],\
            ['0.5+','0.7+','0.8+','0.9+','1','1.4+','2.3+']]
for i,e in features:
    print(i)
    print(e)

age_grouped
exp_grouped
F
M
only_B
other_than_B


ValueError: too many values to unpack (expected 2)